# Project 1: Explanatory Data Analysis & Data Presentation (Movies Dataset)

## Data Import and first Inspection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df = pd.read_csv("movies_complete.csv", parse_dates= ["release_date"])

In [ ]:
df

In [ ]:
df.info()

__Features__:

* **id:** The ID of the movie (clear/unique identifier).
* **title:** The Official Title of the movie.
* **tagline:** The tagline of the movie.
* **release_date:** Theatrical Release Date of the movie.
* **genres:** Genres associated with the movie.
* **belongs_to_collection:** Gives information on the movie series/franchise the particular film belongs to.
* **original_language:** The language in which the movie was originally shot in.
* **budget_musd:** The budget of the movie in million dollars.
* **revenue_musd:** The total revenue of the movie in million dollars.
* **production_companies:** Production companies involved with the making of the movie.
* **production_countries:** Countries where the movie was shot/produced in.
* **vote_count:** The number of votes by users, as counted by TMDB.
* **vote_average:** The average rating of the movie.
* **popularity:** The Popularity Score assigned by TMDB.
* **runtime:** The runtime of the movie in minutes.
* **overview:** A brief blurb of the movie.
* **spoken_languages:** Spoken languages in the film.
* **poster_path:** The URL of the poster image.
* **cast:** (Main) Actors appearing in the movie.
* **cast_size:** number of Actors appearing in the movie.
* **director:** Director of the movie.
* **crew_size:** Size of the film crew (incl. director, excl. actors).

In [ ]:
df.genres[1]

In [ ]:
df.cast[1]

In [ ]:
df.describe()

In [ ]:
df.hist(figsize = (20, 12), bins = 100)
plt.show()

In [ ]:
df.budget_musd.value_counts(dropna = False).head(20)

In [ ]:
df.revenue_musd.value_counts(dropna = False).head(20)

In [ ]:
df.vote_average.value_counts(dropna = False)

In [ ]:
df.vote_count.value_counts()

In [ ]:
df.describe(include = "object")

In [ ]:
df[df.title == "Cinderella"]

## The best and the worst movies... (Part 1)

In [ ]:
from IPython.display import HTML
pd.options.display.max_colwidth = 200

In [ ]:
df_best = df[["poster_path", "title", "budget_musd", "revenue_musd",
              "vote_count", "vote_average", "popularity"]].copy()
df_best

In [ ]:
df_best["profit_musd"] = df.revenue_musd.sub(df.budget_musd)
df_best["return"] = df.revenue_musd.div(df.budget_musd)

In [ ]:
df_best

In [ ]:
df_best.columns = ["", "Title", "Budget", "Revenue", "Votes", 
                   "Average Rating", "Popularity", "Profit", "ROI"]

In [ ]:
df_best.set_index("Title", inplace = True)

In [ ]:
df_best

In [ ]:
df_best.iloc[0,0]

In [ ]:
subset = df_best.iloc[:5, :2]
subset

In [ ]:
HTML(subset.to_html(escape=False))

In [ ]:
df_best.sort_values(by = "Average Rating", ascending = False)

In [ ]:
df_best.sort_values(by = "ROI", ascending = False)

In [ ]:
df_best.loc[df_best.Budget >= 5].sort_values(by = "ROI", ascending = False)

In [ ]:
df_best.Budget.fillna(0, inplace = True)
df_best.Votes.fillna(0, inplace = True)

In [ ]:
df_best.info()

In [ ]:
def best_worst(n, by, ascending = False, min_bud = 0, min_votes = 0):
    
    
    df2 = df_best.loc[(df_best.Budget >= min_bud) & (df_best.Votes >= min_votes), 
                      ["", by]].sort_values(by = by, ascending = ascending).head(n).copy()
    
    return HTML(df2.to_html(escape=False))

## The best and the worst movies... (Part 2)

__Movies Top 5 - Highest Revenue__

In [ ]:
best_worst(n = 5, by = "Revenue")

__Movies Top 5 - Highest Budget__

In [ ]:
best_worst(5, "Budget")

__Movies Top 5 - Highest Profit__

In [ ]:
best_worst(5, "Profit")

__Movies Top 5 - Lowest Profit__

In [ ]:
best_worst(5, "Profit", ascending = True)

__Movies Top 5 - Highest ROI__

In [ ]:
best_worst(5, "ROI", min_bud = 50)

__Movies Top 5 - Lowest ROI__

In [ ]:
best_worst(5, "ROI", ascending = True, min_bud = 100)

__Movies Top 5 - Most Votes__

In [ ]:
best_worst(5, "Votes")

__Movies Top 5 - Highest Rating__

In [ ]:
best_worst(5, "Average Rating", min_votes = 50)

__Movies Top 5 - Lowest Rating__

In [ ]:
best_worst(5, "Average Rating", ascending = True, min_votes = 100)

In [ ]:
best_worst(5, "Average Rating", ascending = True, min_votes = 20, min_bud = 20)

__Movies Top 5 - Most Popular__

In [ ]:
best_worst(5, "Popularity")

## Find your next Movie

__Search 1: Science Fiction Action Movie with Bruce Willis (high Rating)__

In [ ]:
df.genres[0]

In [ ]:
mask_genres = df.genres.str.contains("Action") & df.genres.str.contains("Science Fiction")
mask_genres

In [ ]:
df.cast[0]

In [ ]:
mask_actor = df.cast.str.contains("Bruce Willis")
mask_actor

In [ ]:
df.loc[mask_actor & mask_genres, ["title", "vote_average"]].sort_values(by = "vote_average", 
                                                                        ascending = False)

In [ ]:
bruce = df.loc[mask_actor & mask_genres, ["title", "poster_path", "vote_average"]].sort_values(by = "vote_average", ascending = False).set_index("title")

In [ ]:
HTML(bruce.to_html(escape=False))

__Search 2: Movies with Uma Thurman and directed by Quentin Tarantino (low runtime)__

In [ ]:
df.director

In [ ]:
mask_director = df.director == "Quentin Tarantino"

In [ ]:
mask_actor = df.cast.str.contains("Uma Thurman")

In [ ]:
quentin = df.loc[mask_director & mask_actor, 
               ["title", "poster_path", "runtime"]].sort_values(by = "runtime").set_index("title")

In [ ]:
HTML(quentin.to_html(escape=False))

__Search 3: Most Successful Pixar Studio Movies between 2010 and 2015 (high Revenue)__

In [ ]:
df.production_companies[1]

In [ ]:
mask_studio = df.production_companies.str.contains("Pixar").fillna(False)

In [ ]:
df.release_date

In [ ]:
mask_time = df.release_date.between("2010-01-01", "2015-12-31")

In [ ]:
pixar = df.loc[mask_studio & mask_time, 
               ["title", "poster_path", "revenue_musd", "release_date"]].sort_values(by = "revenue_musd", ascending = False).set_index("title")

In [ ]:
HTML(pixar.to_html(escape=False))

__Search 4: Action or Thriller Movie with original language English and minimum Rating of 7.5 (most recent)__

In [ ]:
mask_genre = df.genres.str.contains("Action") | df.genres.str.contains("Thriller")

In [ ]:
mask_lan = df.original_language == "en"

In [ ]:
mask_vote_av = df.vote_average >= 7.5 

In [ ]:
mask_vote_co = df.vote_count >= 10

In [ ]:
next_mov = df.loc[mask_genre & mask_lan & mask_vote_av & mask_vote_co, 
               ["title", "poster_path", "genres", "vote_average", "vote_count", "release_date"]].sort_values(by = "release_date", ascending = False).set_index("title").head(20)

In [ ]:
HTML(next_mov.to_html(escape=False))

## What are the most common Words in Movie Titles and Taglines?

In [ ]:
from wordcloud import WordCloud

In [ ]:
df

In [ ]:
df.tagline[1]

In [ ]:
df.overview[1]

In [ ]:
title = df.title.dropna()
overview = df.overview.dropna()
tagline = df.tagline.dropna()

In [ ]:
title

In [ ]:
' '.join(title)

In [ ]:
title_corpus = ' '.join(title)
overview_corpus = ' '.join(overview)
tagline_corpus = ' '.join(tagline)

In [ ]:
tagline_corpus

In [ ]:
title_wordcloud = WordCloud(background_color='white', height=2000, width=4000, max_words= 200).generate(title_corpus)
title_wordcloud

In [ ]:
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()

In [ ]:
tagline_wordcloud = WordCloud(background_color='white', height=2000, width=4000).generate(tagline_corpus)
plt.figure(figsize=(16,8))
plt.imshow(tagline_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()

In [ ]:
overview_wordcloud = WordCloud(background_color='white', height=2000, width=4000).generate(overview_corpus)
plt.figure(figsize=(16,8))
plt.imshow(overview_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()

## Are Franchises more successful?

In [ ]:
df.belongs_to_collection

In [ ]:
df["Franchise"] = df.belongs_to_collection.notna()

In [ ]:
df.Franchise

In [ ]:
df.Franchise.value_counts()

__Franchise vs. Stand-alone: Average Revenue__

In [ ]:
df.groupby("Franchise").revenue_musd.mean()

__Franchise vs. Stand-alone: Return on Investment / Profitability__

In [ ]:
df["ROI"] = df.revenue_musd.div(df.budget_musd)

In [ ]:
df.groupby("Franchise").ROI.median()

__Franchise vs. Stand-alone: Average Budget__

In [ ]:
df.groupby("Franchise").budget_musd.mean()

__Franchise vs. Stand-alone: Average Popularity__

In [ ]:
df.groupby("Franchise").popularity.mean()

__Franchise vs. Stand-alone: Average Rating__

In [ ]:
df.groupby("Franchise").vote_average.mean()

In [ ]:
df.groupby("Franchise").agg({"budget_musd": "mean", "revenue_musd": "mean", "vote_average": "mean",
                            "popularity": "mean", "ROI":"median", "vote_count":"mean"})

## Most Successful Franchises

In [ ]:
df.belongs_to_collection

In [ ]:
df.belongs_to_collection.value_counts()

In [ ]:
franchises = df.groupby("belongs_to_collection").agg({"title":"count", "budget_musd": ["sum", "mean"], 
                                                      "revenue_musd": ["sum", "mean"],
                                                      "vote_average": "mean", "popularity": "mean",
                                                      "ROI":"median", 
                                                      "vote_count":"mean"})

In [ ]:
franchises

In [ ]:
franchises.nlargest(20, ("title", "count"))

In [ ]:
franchises.nlargest(20, ("revenue_musd", "mean"))

In [ ]:
franchises.nlargest(20, ("budget_musd", "mean"))

In [ ]:
franchises[franchises[("vote_count", "mean")] >=1000].nlargest(20, ("vote_average", "mean"))

## Most Successful Directors

In [ ]:
df.director

In [ ]:
df.director.value_counts().head(20)

In [ ]:
plt.figure(figsize = (12, 8))
df.director.value_counts().head(20).plot(kind='bar', fontsize = 15)
plt.title("Most Active Directors",fontsize = 20)
plt.ylabel("Number of Movies", fontsize = 15)
plt.show()

In [ ]:
df.groupby("director").revenue_musd.sum().nlargest(20)

In [ ]:
plt.figure(figsize = (12, 8))
df.groupby("director").revenue_musd.sum().nlargest(20).plot(kind='bar', fontsize = 15)
plt.title("Total Revenue",fontsize = 20)
plt.ylabel("Revenue (in MUSD)", fontsize = 15)
plt.show()

In [ ]:
directors = df.groupby("director").agg({"title": "count", "vote_average" :"mean", "vote_count": "sum"})

In [ ]:
directors

In [ ]:
directors[(directors.vote_count >= 10000) & (directors.title >= 10)].nlargest(20, "vote_average")

In [ ]:
df.genres = df.genres.astype(str)

In [ ]:
df.loc[df.genres.str.contains("Horror")].groupby("director").revenue_musd.sum().nlargest(20)

## Most Successful Actors (Part 1)

In [ ]:
df.cast

In [ ]:
df

In [ ]:
df.set_index("id", inplace = True)

In [ ]:
df.info()

In [ ]:
df.cast

In [ ]:
df.cast.str.split("|", expand = True)

In [ ]:
act = df.cast.str.split("|", expand = True)
act

In [ ]:
act.stack().reset_index(level=1, drop=True).to_frame()

In [ ]:
act = act.stack().reset_index(level=1, drop=True).to_frame()

In [ ]:
act

In [ ]:
act.columns = ["Actor"]

In [ ]:
act = act.merge(df[["title", "revenue_musd", "vote_average", "popularity"]],
                how = "left", left_index = True, right_index = True)

In [ ]:
act

## Most Successful Actors (Part 2)

In [ ]:
act

In [ ]:
act.Actor.nunique()

In [ ]:
act.Actor.unique()

In [ ]:
act.Actor.value_counts().head(20)

In [ ]:
plt.figure(figsize = (12, 8))
act.Actor.value_counts().head(20).plot(kind='bar', fontsize = 15)
plt.title("Most Active Actors",fontsize = 20)
plt.ylabel("Number of Movies", fontsize = 15)
plt.show()

In [ ]:
agg = act.groupby("Actor").agg(Total_Revenue = ("revenue_musd", "sum"), 
                               Mean_Revenue = ("revenue_musd", "mean"),
                               Mean_Rating = ("vote_average", "mean"), 
                               Mean_Pop = ("popularity", "mean"), 
                               Total_Movies = ("Actor", "count"))

In [ ]:
agg.nlargest(10, "Total_Movies")

In [ ]:
agg.nlargest(10, "Total_Revenue")

In [ ]:
plt.figure(figsize = (12, 8))
agg.Total_Revenue.nlargest(10).plot(kind='bar', fontsize = 15)
plt.title("Total Revenue",fontsize = 20)
plt.ylabel("Revenue (in MUSD)", fontsize = 15)
plt.show()

In [ ]:
agg.Mean_Revenue.nlargest(10)

In [ ]:
act[act.Actor == "Ashley Jeffery"]

In [ ]:
agg[agg.Total_Movies >= 10].nlargest(10, "Mean_Revenue")

In [ ]:
agg[agg.Total_Movies >= 10].nlargest(10, "Mean_Rating")

In [ ]:
agg[agg.Total_Movies >= 10].nlargest(10, "Mean_Pop")